<a href="https://colab.research.google.com/github/dhanalakshmi3149/diabatics/blob/main/Copy_of_Sentiment_Analysis_using_Pypark_NLP_Week_10_type01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.4.1
!pip install spark-nlp==5.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 688.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=e8ee4da240db8666ad69721004b4ed1e658057ec09162efc6b8d0335008f7497
  Stored in directory: /root/.cache/pip/wheels/e9/b4/d8/38accc42606f6675165423e9f0236f8e825f6b6b6048d6743e
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 1

In [2]:
from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, BertEmbeddings, SentimentDLModel, LemmatizerModel, UniversalSentenceEncoder
from pyspark.ml import Pipeline as MLPipeline

# Start Spark NLP session
import sparknlp
spark = sparknlp.start()

# Sample dataset (50 sample sentences)
sentences = [
    "I love this movie!", "This was the worst experience.", "Pretty decent overall.",
    "Absolutely fantastic!", "I'm not sure how I feel.", "Worst purchase ever.",
    "Great value for the money.", "It was okay, not great.", "Terrible, just terrible.",
    "Super fun and engaging!", "Would not recommend it.", "Kind of boring.",
    "Loved every minute!", "It was a disaster.", "Highly recommend!",
    "Too expensive for what you get.", "Amazing support team!", "Horrible food.",
    "I'll definitely buy it again.", "Meh, nothing special.", "Exceeded my expectations!",
    "Not worth the hype.", "Incredible storytelling.", "Never again.",
    "Pretty enjoyable!", "Worst customer service.", "Delightful and fresh.",
    "Disappointed.", "Can't wait to try it again!", "A total waste of time.",
    "Superb work!", "Very underwhelming.", "Loved the packaging.",
    "Felt very rushed.", "Unbelievably good.", "Didn't like it at all.",
    "Perfect execution!", "Mediocre at best.", "Simply amazing.",
    "It made my day!", "Forgettable.", "Super smooth experience.",
    "I regret buying it.", "Truly inspiring.", "Boring and repetitive.",
    "Five stars!", "Nothing new.", "Highly entertaining.", "Wasted potential.", "It was alright."
]

# Create DataFrame
df = spark.createDataFrame([(s,) for s in sentences], ["text"])

# Spark NLP pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

# Add sentence embeddings
use = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

# Use pretrained sentiment model (Twitter-based)
sentiment_model = SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

# Create pipeline
pipeline = Pipeline(stages=[document_assembler, use, sentiment_model])

# Run the pipeline
result = pipeline.fit(df).transform(df)

# Show results
result.select("text", "sentiment.result").show(truncate=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]
+-------------------------------+----------+
|text                           |result    |
+-------------------------------+----------+
|I love this movie!             |[positive]|
|This was the worst experience. |[negative]|
|Pretty decent overall.         |[positive]|
|Absolutely fantastic!          |[positive]|
|I'm not sure how I feel.       |[negative]|
|Worst purchase ever.           |[negative]|
|Great value for the money.     |[positive]|
|It was okay, not great.        |[neutral] |
|Terrible, just terrible.       |[negative]|
|Super fun and engaging!        |[positive]|
|Would not recommend it.        |[negative]|
|Kind of boring.                |[negative]|
|Loved every minute!            |[positive]|
|It was a disaster.             |[negative]|
|Highly recommend!            